In [16]:
# Install build tools
!apt-get install -y build-essential
!pip install Cython

# Downgrade NumPy and reinstall Surprise
!pip uninstall -y numpy
!pip install numpy==1.26.4
!pip install scikit-surprise --no-cache-dir

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)


In [17]:
!pip install pyngrok
!pip install streamlit
!ngrok config add-authtoken 2tvhE6Q6TIIvQl5kkMFazgSft6d_4vHmveTfcneMahtTeX6A9

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [18]:
# Import necessary libraries
import os
import pickle
import pandas as pd
from flask import Flask, request, jsonify
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from pyngrok import ngrok
import warnings
warnings.filterwarnings('ignore')

# Import Libraries
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
import joblib
import pickle
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from flask import Flask, request, jsonify
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from surprise import Dataset, Reader, SVD, KNNBasic, SVDpp
from surprise.model_selection import cross_validate
warnings.filterwarnings('ignore')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [ ]:
# Importing necessary libraries
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load dataset
df = pd.read_csv("/content/drive/MyDrive/Master Degree Capstone/Travel Prediction/hotels.csv",on_bad_lines='skip')

In [ ]:
hotels_df = df

In [ ]:
hotels_df.head()

In [ ]:
hotels_df.nunique()

In [ ]:
hotels_df.shape

In [ ]:
hotels_df.info()

In [ ]:
hotels_df.duplicated().sum()

In [ ]:
hotels_df.isnull().sum()

In [ ]:
hotels_df.columns

In [ ]:
hotels_df.describe()

In [ ]:
hotels_df.nunique()

In [ ]:
label_encoders = {}
for column in ['name', 'place']:
    le = LabelEncoder()
    hotels_df[column] = le.fit_transform(hotels_df[column])
    label_encoders[column] = le

In [ ]:
hotels_df['date'] = pd.to_datetime(hotels_df['date'])
hotels_df['day_of_week'] = hotels_df['date'].dt.dayofweek
hotels_df['month'] = hotels_df['date'].dt.month

In [ ]:
hotels_df = hotels_df.drop(columns=['date'])

In [ ]:
hotels_df = hotels_df.drop(columns=['travelCode'])
# hotels_df = hotels_df.drop(columns=['userCode'])

In [ ]:
hotels_df.head()

In [ ]:
train_df, test_df = train_test_split(hotels_df, test_size=0.2, random_state=42)

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(hotels_df[['userCode', 'name', 'price']], reader)
data

In [ ]:
# Build and evaluate the SVD model
algo = SVD()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Train the model on the entire dataset
trainset = data.build_full_trainset()
algo.fit(trainset)

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(algo, f)

In [ ]:
from google.colab import files

# Download the model.pkl file
files.download('model.pkl')

In [ ]:
from pyngrok import ngrok

# Set your ngrok auth token
NGROK_AUTH_TOKEN = '2tvhE6Q6TIIvQl5kkMFazgSft6d_4vHmveTfcneMahtTeX6A9'
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok tunnel to expose port 8501
public_url = ngrok.connect(8501)
print('Public URL:', public_url)

streamlit_script = f"""
import streamlit as st
import pandas as pd
import joblib

# Load the SVD model
svd_model = joblib.load('model.pkl')

# Load the dataset
hotels_df = pd.read_csv('/content/drive/MyDrive/Master Degree Capstone/Travel Prediction/hotels.csv')

# Define your recommendation function with filters
def get_recommendations(user_code, place, model, data, min_price, max_price, sort_option):
    # Filter hotels based on the input place and price range
    place_filtered_df = data[(data['place'].str.lower() == place.lower()) &
                             (data['price'] >= min_price) & (data['price'] <= max_price)]

    if place_filtered_df.empty:
        return pd.DataFrame()  # Return an empty DataFrame if no hotels are found for the place

    # Generate predictions for each hotel in the filtered list
    hotel_ids = place_filtered_df['userCode'].unique()  # Assuming 'userCode' is the hotel/item ID

    predictions = []
    for hotel_id in hotel_ids:
        # Predict the rating for each hotel using the model
        prediction = model.predict(uid=user_code, iid=hotel_id)
        predictions.append((hotel_id, prediction.est))  # 'est' is the estimated rating

    # Sort the hotels by predicted rating
    top_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]  # Top 5 recommendations

    # Get the top hotels' details
    top_hotel_ids = [pred[0] for pred in top_predictions]
    top_hotels = place_filtered_df[place_filtered_df['userCode'].isin(top_hotel_ids)]

    # Apply sorting based on the selected option
    if sort_option == "Low to High":
        top_hotels = top_hotels.sort_values(by='price', ascending=True)
    elif sort_option == "High to Low":
        top_hotels = top_hotels.sort_values(by='price', ascending=False)

    # Define the columns we want to display, checking if they exist
    columns_to_display = ['name', 'place', 'price', 'days', 'total']

    if 'day_of_week' in top_hotels.columns:
        columns_to_display.append('day_of_week')
    if 'month' in top_hotels.columns:
        columns_to_display.append('month')

    # Return the selected columns for display
    return top_hotels[columns_to_display]

# Streamlit app interface
st.title("Hotel Recommendation System")

# Create a dropdown for the available places
available_places = hotels_df['place'].unique().tolist()
place = st.selectbox("Select Destination Place:", available_places)

# Create user input for recommendations
user_code = st.number_input("Enter User Code:", min_value=1, max_value=hotels_df['userCode'].max())

# Price range filter
min_price = st.number_input("Minimum Price:", min_value=0, value=int(hotels_df['price'].min()))
max_price = st.number_input("Maximum Price:", min_value=0, value=int(hotels_df['price'].max()))

# Sorting option
sort_option = st.selectbox("Sort by Price:", ["None", "Low to High", "High to Low"])

if st.button("Recommend Hotels"):
    recommendations = get_recommendations(user_code, place, svd_model, hotels_df, min_price, max_price, sort_option)
    if recommendations.empty:
        st.write("No recommendations found for the specified place.")
    else:
        st.write("Recommended Hotels:")
        st.write(recommendations)
"""

with open('app.py', 'w') as f:
    f.write(streamlit_script)

# Run your Streamlit app on port 8501
!streamlit run app.py --server.port 8501 &

# Keep the cell running to keep the tunnel alive
import time
while True:
    time.sleep(100)